In [1]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
import matplotlib.pyplot as plt
import numpy as np
import googlemaps

In [2]:
df = pd.read_csv('Original-Data.csv')

In [4]:
pd.set_option('display.max_columns', None)

# df

In [5]:
# Check for any missing values and handle them if necessary
print(df.isnull().sum())

# If there are missing values, you can handle them using methods like fillna() or dropna()

# Count the different types in the "Type" column
type_counts = df['Type'].value_counts()

# Display the counts of different types
print()
print(type_counts)


Date                         0
Price                        0
Status                       0
Build Year                   0
URL                          0
Adresse                      0
Kort adresse                 0
Kvdm                         9
Latitude                   620
Longitude                  620
Kommune                      0
Type                         0
Nearest_Metro_Distance    6686
Nearest_Tog_Distance      6686
dtype: int64

Type
Ejerlejlighed    77328
Villa            33099
Rækkehus         14958
Name: count, dtype: int64
